In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Check TensorFlow version
print(tf.__version__)

# Path to your dataset
DATASET_PATH = '../dataset/'  # adjust if needed

# Image preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% train, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=16,
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=16,
    subset='validation'
)

# Load base MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False  # freeze base layers

# Build the model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 car classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

# Save the full model
model.save('car_model.h5')

# Export to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('car_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ Model trained and saved!")


ModuleNotFoundError: No module named 'tensorflow'

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="car_model.tflite")
interpreter.allocate_tensors()

# Load the image
img = image.load_img('test.jpg', target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale

# Set up model input
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()

# Get prediction
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_index = np.argmax(output_data)

# Load labels
with open('labels.text', 'r') as f:
    labels = [line.strip() for line in f.readlines()]

print(f"Prediction: {labels[predicted_index]}")


Prediction: Ford GT
